In [43]:
# Dependencies
import json
import requests
import pandas as pd
import plotly.offline as off
import plotly.plotly as py
off.init_notebook_mode(connected=True)
from pytrends.request import TrendReq

# Google Developer API Key
from config import gkey

In [44]:
# Census Data Collected from csv's
income = pd.read_csv("Resources/income.csv")
age = pd.read_csv("Resources/age.csv")

In [45]:
# Setup Dataframe to Collect Data from CSVs
stats_df = pd.DataFrame()

stats_df["City-State"] = age["Geography"]
stats_df["Total Population"] = age["Total; Estimate; Total population"]
stats_df["Target Population"] = age["Total; Estimate; AGE - 20 to 24 years"] + age["Total; Estimate; AGE - 25 to 29 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 35 to 39 years"]
stats_df["Median Income"] = income["Median income (dollars); Estimate; HOUSEHOLD INCOME BY AGE OF HOUSEHOLDER - 25 to 44 years"].map(
    "${:,.2f}".format)
#stats_df.head()

In [46]:
# Build Keywords from Suggested Search
keywords=[]
pytrend = TrendReq(hl='en-US')
suggestions = pytrend.suggestions(keyword = 'Brewery')
for item in range(0,len(suggestions)):
    keywords.append(suggestions[item]['title'])

In [47]:
# List for Data Collection
lat = []
long = []
results = []

# Create Dataframe for API Calls
city_df = stats_df
city_df["City-URL-String"] = city_df["City-State"].str.replace("Metro Area","")
city_df["City-URL-String"] = city_df["City-URL-String"].str.replace(" ","+")
#city_df

In [48]:
# Collects the API Results for Each City
for row in range(0,len(city_df)):
    city = city_df["City-URL-String"].loc[row]
    
    # Prints what current city is being called by the API
    print(city)
    
    # Create String for API
    url_string_call =  f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    brewery_data = requests.get(url_string_call)
    
    # Formats Latitude and Longitude Data for Second API Call
    test_var = brewery_data.json()
    target_city = test_var["results"][0]['geometry']['location']
    target_coords = f"{target_city['lat']},{target_city['lng']}"
    params = {
        "location": target_coords,
        "radius": 8000,
        "keyword": keywords,
        "key": gkey
    }
    
    # Second API Call
    base_url = "https://maps.googleapis.com/maps/api/place/radarsearch/json"
    brewery_data = requests.get(base_url, params=params)
    brewery_data = brewery_data.json()

    # Saves Results Data into list to be converted into Dataframes 
    results.append(len(brewery_data["results"]))
    print(len(brewery_data["results"]))
    lat.append(target_city['lat'])
    long.append(target_city['lng'])

Abilene,+TX+
0
Akron,+OH+
0
Albany,+GA+
0
Albany,+OR+
0
Albany-Schenectady-Troy,+NY+
0
Albuquerque,+NM+
0
Alexandria,+LA+
0
Allentown-Bethlehem-Easton,+PA-NJ+
0
Altoona,+PA+


KeyboardInterrupt: 

In [38]:
# Add Collected Data to DataFrame
city_df['results']=pd.DataFrame({'results':results})
city_df['lat']=pd.DataFrame({'lat':lat})
city_df['long']=pd.DataFrame({'long':long})

# Create Ratio Column to Weigh Results from Population and Google Results
city_df['Ratio']=city_df['Target Population']/(city_df['results'])

# Remove Data that Has Errors
city_clean_df = city_df[city_df['results'] != 0]
df = city_clean_df
city_clean_df.sort_values(['results'])
city_clean_df

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio
22,"Bakersfield, CA Metro Area",871337,36.8,"$50,957.00","Bakersfield,+CA+",182,35.373292,-119.018713,0.202198
67,"Cheyenne, WY Metro Area",96459,35.4,"$69,092.00","Cheyenne,+WY+",199,41.139981,-104.820246,0.177889
203,"Lexington-Fayette, KY Metro Area",495193,37.4,"$56,366.00","Lexington-Fayette,+KY+",195,38.040584,-84.503716,0.191795
208,"Longview, TX Metro Area",216775,32.6,"$52,529.00","Longview,+TX+",172,32.500704,-94.740489,0.189535
223,"Memphis, TN-MS-AR Metro Area",1341339,33.9,"$48,231.00","Memphis,+TN-MS-AR+",159,35.038725,-89.850501,0.213208
266,"Parkersburg-Vienna, WV Metro Area",92088,28.8,"$46,929.00","Parkersburg-Vienna,+WV+",115,39.282148,-81.537701,0.250435


In [34]:
df['text'] = city_clean_df['City-State'] + '<br>Population ' + (city_clean_df['Total Population']/1e6).astype(str)+' million'

colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 10

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [35]:

df_sub = city_clean_df
df_sub['scale']=((df_sub['Target Population']/scale)*25)
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['scale'],
        color = colors[0],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'City Population')
cities.append(city)


/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [36]:

df_sub = city_clean_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['results'],
        color = colors[1],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Google Map Results')
cities.append(city)


In [37]:

df_sub = city_clean_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['Ratio']*100,
        color = colors[2],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Ratio of Population by Results')
cities.append(city)

layout = dict(
        title = 'USA Brewery Data<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [31]:
top_five_df = city_clean_df.sort_values(['Ratio'], ascending=False).head()
top_five_df

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio,text,scale
266,"Parkersburg-Vienna, WV Metro Area",92088,28.8,"$46,929.00","Parkersburg-Vienna,+WV+",115,39.282148,-81.537701,0.250435,"Parkersburg-Vienna, WV Metro Area<br>Populatio...",72.00
223,"Memphis, TN-MS-AR Metro Area",1341339,33.9,"$48,231.00","Memphis,+TN-MS-AR+",159,35.038725,-89.850501,0.213208,"Memphis, TN-MS-AR Metro Area<br>Population 1.3...",84.75
22,"Bakersfield, CA Metro Area",871337,36.8,"$50,957.00","Bakersfield,+CA+",182,35.373292,-119.018713,0.202198,"Bakersfield, CA Metro Area<br>Population 0.871...",92.00
203,"Lexington-Fayette, KY Metro Area",495193,37.4,"$56,366.00","Lexington-Fayette,+KY+",195,38.040584,-84.503716,0.191795,"Lexington-Fayette, KY Metro Area<br>Population...",93.50
208,"Longview, TX Metro Area",216775,32.6,"$52,529.00","Longview,+TX+",172,32.500704,-94.740489,0.189535,"Longview, TX Metro Area<br>Population 0.216775...",81.50


In [32]:
df['text'] = city_clean_df['City-State'] + '<br>Population ' + (city_clean_df['Total Population']/1e6).astype(str)+' million'

cities = []
df_sub = top_five_df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['scale'],
        color = colors[0],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Top 5 City Population')
cities.append(city)

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['results'],
        color = colors[1],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Google Map Results')
cities.append(city)

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['Ratio']*100,
        color = colors[2],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Ratio of Population by Results')
cities.append(city)

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [33]:
layout = dict(
        title = 'USA Brewery Data<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )